In [ ]:
#====================================================================================================#
#                                                                                                    #
#                                                        ██╗   ██╗   ████████╗ █████╗ ██████╗        #
#      Competición - INAR                                ██║   ██║   ╚══██╔══╝██╔══██╗██╔══██╗       #
#                                                        ██║   ██║█████╗██║   ███████║██║  ██║       #
#      created:        07/11/2025  -  05:21:23           ██║   ██║╚════╝██║   ██╔══██║██║  ██║       #
#      last change:    09/11/2025  -  19:29:12           ╚██████╔╝      ██║   ██║  ██║██████╔╝       #
#                                                         ╚═════╝       ╚═╝   ╚═╝  ╚═╝╚═════╝        #
#                                                                                                    #
#      Ismael Hernandez Clemente                         ismael.hernandez@live.u-tad.com             #
#                                                                                                    #
#      Github:                                           https://github.com/ismaelucky342            #
#                                                                                                    #
#====================================================================================================# 

# Competición Perretes y Gatos

## Iteración 7 - EfficientNetB7

Modelo para mi bastante sobredimensionado, la idea es probar el top de la familia, aunque va a ser eterno el entrenamiento. 

**kaggle score**: 0.94962, mejora elevada pero he sacrificado casi 4h de entrenamiento y el modelo era innecesariamente grande. 


In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time

from tensorflow import data as tf_data
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.applications import EfficientNetB7  

seed = 42
keras.utils.set_random_seed(seed)
np.random.seed(seed)

# Rutas dataset
DATASET_NAME = "u-tad-dogs-vs-cats-2025"
TRAIN_PATH = f"/kaggle/input/{DATASET_NAME}/train/train"
TEST_PATH = f"/kaggle/input/{DATASET_NAME}/test/test"
SUPP_PATH = f"/kaggle/input/{DATASET_NAME}/supplementary_data/supplementary_data"

print("Keras:", keras.__version__)

## Config 

In [ ]:

USE_HIGH_RES = True  # 384x384 
N_TTA_AUGMENTATIONS = 10  # TTA 
DROPOUT_RATE = 0.6  # Regularización fuerte
N_FINE_TUNE_LAYERS = 30  # REDUCIDO de 40 para que acelerar
EPOCHS_TL = 10  # REDUCIDO de 15
EPOCHS_FT = 10  # REDUCIDO de 15

# Parámetros dinámicos
IMG_SIZE = 384 if USE_HIGH_RES else 224
BATCH_SIZE = 16  # Más bajo para que B7 (consume más RAM)

print(f"Modelo: EfficientNetB7")
print(f"Resolución: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"TTA augmentations: {N_TTA_AUGMENTATIONS}")
print(f"Dropout: {DROPOUT_RATE}")
print(f"Fine-tune layers: {N_FINE_TUNE_LAYERS}")
print(f"Épocas TL: {EPOCHS_TL}")
print(f"Épocas FT: {EPOCHS_FT}")

## Carga de Datos

Todo igual q antes, pero con batch_size=16 pa q entre B7 en memoria.

In [ ]:
# Cargo datos con ALTA resolución (384x384)
train_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
)

validation_dataset = keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=True,
    seed=seed,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
)

test_dataset = keras.utils.image_dataset_from_directory(
    TEST_PATH,
    labels=None,
    label_mode=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

supplementary_dataset = keras.utils.image_dataset_from_directory(
    SUPP_PATH,
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE),
    shuffle=False,
    seed=seed,
    interpolation='bilinear',
)

print(f"Train batches: {len(train_dataset)}")
print(f"Validation batches: {len(validation_dataset)}")
print(f"Test batches: {len(test_dataset)}")
print(f"Supplementary batches: {len(supplementary_dataset)}")

## Data Augmentation

Mismo q antes 
- Flip horizontal + vertical
- Rotation ±20%
- Zoom ±20%
- Translation ±15%
- Contrast ±20%
- Brightness ±20%

In [ ]:
# Augmentation para que evitar overfitting
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal_and_vertical"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomTranslation(height_factor=0.15, width_factor=0.15),
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomBrightness(0.2),
], name="data_augmentation_aggressive")

# Aplico augmentation
train_dataset_augmented = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))
validation_dataset_augmented = validation_dataset.map(lambda x, y: (data_augmentation(x, training=True), y))

## EfficientNetB7 - Construcción

**EfficientNetB7 stats:**
- 66M parámetros (B3 tiene 12M)
- Top-1 Accuracy ImageNet: **84.3%** (B3: 81.6%)
- Resolución nativa: 600x600 (aguanta perfectamente 384x384)
- MÁS LENTO pero MÁS PRECISO

Arquitectura:
- EfficientNetB7 base (congelado)
- BatchNormalization
- Dense(512) + Dropout(0.6)
- BatchNormalization
- Dense(256) + Dropout(0.6)
- Dense(1, sigmoid)

In [ ]:
# Cargo EfficientNetB7 pre-entrenado
efficientnet_base = EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg'
)

# Congelo todo inicialmente
for layer in efficientnet_base.layers:
    layer.trainable = False

# Construyo modelo con B7
efficientnet_model = Sequential([
    efficientnet_base,
    keras.layers.BatchNormalization(),  # Regularización
    Dense(512, activation='relu'),  # Capa densa grande
    Dropout(DROPOUT_RATE),  # 0.6
    keras.layers.BatchNormalization(),  # Más regularización
    Dense(256, activation='relu'),  # Capa intermedia
    Dropout(DROPOUT_RATE),  # 0.6
    Dense(1, activation='sigmoid')  # Output binario
])

efficientnet_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

efficientnet_model.summary()
print(f"EfficientNetB7 layers: {len(efficientnet_base.layers)}")
print(f"Trainable layers: {sum([layer.trainable for layer in efficientnet_base.layers])}/{len(efficientnet_base.layers)}")
print(f"Total parameters: {efficientnet_model.count_params():,}")

## Transfer Learning

Entreno solo las capas Dense q añadí, con EfficientNetB7 congelado.

**15 épocas** con early stopping (patience=5).

In [ ]:
# Transfer Learning con EfficientNetB7
print("Starting Transfer Learning...")

efficientnet_history_tl = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=EPOCHS_TL,
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7
        )
    ]
)

print("Transfer Learning completed")

## Visualización TL

In [ ]:
# Gráficos Transfer Learning (robustos a nombres de métricas)
try:
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(efficientnet_history_tl.history.get('loss', []), label='Train')
    plt.plot(efficientnet_history_tl.history.get('val_loss', []), label='Validation')
    plt.title('Loss - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(efficientnet_history_tl.history.get('accuracy', []), label='Train')
    plt.plot(efficientnet_history_tl.history.get('val_accuracy', []), label='Validation')
    plt.title('Accuracy - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Detect keys for precision/recall (Keras names may vary)
    precision_key = next((k for k in ['precision_1', 'precision'] if k in efficientnet_history_tl.history), None)
    recall_key = next((k for k in ['recall_1', 'recall'] if k in efficientnet_history_tl.history), None)

    plt.subplot(1, 3, 3)
    if precision_key and recall_key:
        plt.plot(efficientnet_history_tl.history[precision_key], label='Precision')
        plt.plot(efficientnet_history_tl.history[recall_key], label='Recall')
    else:
        plt.plot([], [], label='Precision (n/a)')
        plt.plot([], [], label='Recall (n/a)')

    plt.title('Precision & Recall - Transfer Learning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Warning: Plotting failed ({e}). Continuing...")

## Evaluación Supplementary (post-TL)

Veo cómo va ANTES del fine-tuning.

In [ ]:
print("Evaluating on supplementary dataset (post-TL)...")

efficientnet_supp_results_tl = efficientnet_model.evaluate(supplementary_dataset, verbose=1)
efficientnet_supp_accuracy_tl = efficientnet_supp_results_tl[1]

print(f"Supplementary Accuracy (TL): {efficientnet_supp_accuracy_tl:.4f}")

## Fine-tuning - DESCONGELAR 30 CAPAS

Ahora descongelo las **últimas 30 capas** de B7 (optimizado pa velocidad).

LR MUY bajo (5e-6) pa no romper lo pre-entrenado.

In [ ]:
# Descongelo últimas capas de B7
print(f"Unfreezing last {N_FINE_TUNE_LAYERS} layers...")

for layer in efficientnet_base.layers[-N_FINE_TUNE_LAYERS:]:
    layer.trainable = True

# Recompilo con LR MUY bajo
efficientnet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-6),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

trainable_count = sum([layer.trainable for layer in efficientnet_base.layers])
print(f"Trainable layers: {trainable_count}/{len(efficientnet_base.layers)}")

In [ ]:
# Fine-tuning con B7
print("Starting Fine-tuning...")

efficientnet_history_ft = efficientnet_model.fit(
    train_dataset_augmented,
    epochs=EPOCHS_FT,
    validation_data=validation_dataset_augmented,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-8
        )
    ]
)

print("Fine-tuning completed")

## Visualización Fine-tuning

In [ ]:
try:
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 3, 1)
    plt.plot(efficientnet_history_ft.history.get('loss', []), label='Train')
    plt.plot(efficientnet_history_ft.history.get('val_loss', []), label='Validation')
    plt.title('Loss - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 3, 2)
    plt.plot(efficientnet_history_ft.history.get('accuracy', []), label='Train')
    plt.plot(efficientnet_history_ft.history.get('val_accuracy', []), label='Validation')
    plt.title('Accuracy - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Detect keys for precision/recall (Keras names may vary)
    precision_key_ft = next((k for k in ['precision_1', 'precision'] if k in efficientnet_history_ft.history), None)
    recall_key_ft = next((k for k in ['recall_1', 'recall'] if k in efficientnet_history_ft.history), None)

    plt.subplot(1, 3, 3)
    if precision_key_ft and recall_key_ft:
        plt.plot(efficientnet_history_ft.history[precision_key_ft], label='Precision')
        plt.plot(efficientnet_history_ft.history[recall_key_ft], label='Recall')
    else:
        plt.plot([], [], label='Precision (n/a)')
        plt.plot([], [], label='Recall (n/a)')

    plt.title('Precision & Recall - Fine-tuning B7')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Warning: Plotting failed ({e}). Continuing...")

## TTA - Test Time Augmentation

**10 augmentations**

Con B7 va a tardar MÁS (cada predicción es más lenta), pero merece la pena.

In [ ]:
def predict_with_tta(model, img_array, n_augmentations=10):
    """TTA con augmentations agresivos"""
    augmentation_layer = keras.Sequential([
        keras.layers.RandomFlip("horizontal_and_vertical"),
        keras.layers.RandomRotation(0.2),
        keras.layers.RandomZoom(0.2),
        keras.layers.RandomTranslation(0.15, 0.15),
        keras.layers.RandomContrast(0.2),
        keras.layers.RandomBrightness(0.2),
    ])
    
    predictions = []
    for _ in range(n_augmentations):
        augmented_img = augmentation_layer(img_array, training=True)
        pred = model.predict(augmented_img, verbose=0)
        predictions.append(pred)
    
    return np.mean(predictions, axis=0)


## Predicciones Finales con TTA

In [ ]:
print(f"Model: EfficientNetB7")
print(f"TTA augmentations: {N_TTA_AUGMENTATIONS}")
print(f"Resolution: {IMG_SIZE}x{IMG_SIZE}")

predictions = []

for i, batch in enumerate(test_dataset):
    if i % 10 == 0:
        print(f"Processing batch {i+1}/{len(test_dataset)}...")
    pred = predict_with_tta(efficientnet_model, batch, N_TTA_AUGMENTATIONS)
    predictions.extend(pred)

predictions = np.array(predictions)
print(f"Total predictions: {len(predictions)}")

## Generación Submission Final

In [ ]:
# Genero submission.csv
test_filenames = test_dataset.file_paths
ids = [int(os.path.splitext(os.path.basename(f))[0]) for f in test_filenames]

predictions_binary = (predictions > 0.5).astype(int).flatten()

submission_df = pd.DataFrame({
    'id': ids,
    'label': predictions_binary
})

submission_df = submission_df.sort_values('id')
submission_df.to_csv('submission.csv', index=False)
print("Submission saved to submission.csv")
print(submission_df.head(10))